In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
# uiucData = pd.read_csv('uiuc-gpa-dataset.csv')
cs_courses_only = pd.read_csv('cs-uiuc-course-data.csv', index_col=False)
cs_courses_only

In [ ]:
agg_cs_courses = pd.read_csv('agg-cs-uiuc-course-data.csv', index_col=False)
agg_cs_courses

In [ ]:
gpaMappings = { 
    'A+': 4, 
    'A': 4, 
    'A-': 3.67, 
    'B+': 3.33, 
    'B': 3, 
    'B-': 2.67, 
    'C+': 2.33, 
    'C': 2, 
    'C-': 1.67, 
    'D+': 1.33, 
    'D': 1,
    'D-': 0.67,
    'F': 0,
} # W is not calculated in GPA

letters = ['A+', 'A', 'A-', 'B+', 'B', 'B-', 'C+', 'C', 'C-', 'D+', 'D', 'D-', 'F', 'W']

In [ ]:
def averageGPA(row):
    sumGrade = 0.0
    count = 0.0
    for letter in gpaMappings:
        count += row[letter] 
        sumGrade += row[letter] * gpaMappings[letter]
    return round(sumGrade / count, 3)

def studentCount(row):
    count = row['W']
    for letter in gpaMappings:
        count += row[letter]
    return count
    
def getCourseDesc(subject, number, term, year):
    baseUrl = 'https://courses.illinois.edu/schedule'
    completeUrl = baseUrl + '/{}/{}/{}/{}'.format(year, term, subject, number) # /2020/spring/AAS/100'
    coursePage = requests.get(completeUrl)
    assert(coursePage.status_code == 200)
    print(coursePage.url)
    
    pageSoup = BeautifulSoup(coursePage.content, 'html.parser')
    try:
        courseInfo = pageSoup.find(id='app-course-info')
        cols = courseInfo.find_all(class_='col-sm-12')
        lastCol = cols[-1]
        paras = lastCol.find_all('p')
        desc = paras[1].text
        preReqIdx = desc.find('Prerequisite:')
        sameAsIdx = desc.find('Same as')
        cutoffIdx = min(preReqIdx, sameAsIdx)
        if preReqIdx == -1 and sameAsIdx == -1:
            cutoffIdx = len(desc) # no cutoff
        elif preReqIdx == -1:
            cutoffIdx = sameAsIdx # stop at 'Same as'
        elif sameAsIdx == -1:
            cutoffIdx = preReqIdx # stop at 'Prerequisite:'
        # NOTE: there are additional irrelevant portions (i.e. other than the two filtered out above)
        # in some course descriptions that are harder to auto-filter out.
        onlyDesc = desc[:cutoffIdx]
        return onlyDesc
    except:
        print("Failed.")
        return ""

In [ ]:
# allCourses = set()
# for idx, row in uiucData.iterrows():
#     allCourses.add(",".join([ row['Subject'], str(row['Number']), row['Term'], str(row['Year']) ]))
    
# print(len(allCourses))

# i = 0
# uiucData['courseDesc'] = ""
# for course in allCourses:
#     print(i)
#     subject, number, term, year = course.split(",")
#     number = int(number)
#     year = int(year)
#     courseDesc = getCourseDesc(subject, number, term, year)
#     uiucData.loc[(uiucData['Subject'] == subject) & (uiucData['Number'] == number) & (uiucData['Term'] == term) & (uiucData['Year']), 'courseDesc'] = courseDesc
#     i += 1

In [ ]:
# allCourses = set()
# for idx, row in ud.iterrows():
#     allCourses.add(",".join([ row['Subject'], str(row['Number']) ]))
    
# for course in allCourses:
#     subject, number = course.split(",")
#     withDesc = ud[ (ud['Subject'] == subject) & (ud['Number'] == int(number)) & (ud['courseDesc'].str.len() > 0) ].reset_index()
#     protoDesc = withDesc[ withDesc['courseDesc'].str.len() == withDesc['courseDesc'].str.len().max() ]['courseDesc']
#     if len(protoDesc) == 0:
#         ud.loc[ (ud['Subject'] == subject) & (ud['Number'] == int(number)), 'courseDesc' ] = ""
#     else:
#         ud.loc[ (ud['Subject'] == subject) & (ud['Number'] == int(number)), 'courseDesc' ] = protoDesc.iloc[0]

# ud.to_csv('uiuc-gpa-data-with-desc.csv', index=False)
# ud = pd.read_csv('uiuc-gpa-data-with-desc.csv', index_col=False)
# cs_courses_only = ud[ ud['Subject'] == 'CS'].reset_index(drop=True)

In [ ]:
# cs_courses_only = cs_courses_only[cs_courses_only['Term'] != 'Summer']
# cs_courses_only.to_csv('cs-uiuc-course-data.csv', index=False)

In [ ]:
# agg_dict = {}
# for lt in letters:
#     agg_dict[lt] = 'sum'

# agg_dict['courseDesc'] = 'first'
# agg_cs_courses = cs_courses_only.groupby(['Subject', 'Number', 'Course Title', 'Primary Instructor']).agg(agg_dict).reset_index()

# agg_cs_courses['averageGPA'] = agg_cs_courses.apply(averageGPA, axis=1)
# agg_cs_courses['studentCount'] = agg_cs_courses.apply(studentCount, axis=1)
# agg_cs_courses.to_csv('agg-cs-uiuc-course-data.csv', index=False)
# agg_cs_courses